In [1]:
!pip install -q openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.4 MB/s eta 0:00:00


In [ ]:
# For transformer models
!pip install -q accelerate
# !pip install -q bitsandbytes
!pip install -i https://pypi.org/simple/ bitsandbytes
# !pip install -q flash-attn --no-build-isolation

# For sentence similarity
!pip install sentence_transformers

# For web queries
!pip install googlesearch-python

# For Retrieval Augmentated Generation (RAG) since HF doesn't have great support for it
!pip install langchain
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [ ]:
from openai import OpenAI
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
client = OpenAI(
    api_key = OPENAI_API_KEY
)

In [ ]:
claim_atomization_template = """
You are a helpful assistant. Your task is to break down a set of statements given after <<<>>> into a minimal number of atomic claims.
These atomic claims need to be comprehensible, coherent, and context-independent.

Segmentation Criteria:
1. Each atomic claim should focus on a single idea or concept regarding the truthfulness and/or plausibility of the statement.
   - e.g., Gen Z is divided 50-50 on the "issue" -> specify what "issue" is divisive
2. Atomic claims should be independent of each other and not rely heavily on the context of the original statement.
3. Aim for clarity and coherence in the segmented atomic claims.
4. If a statement cannot be broken down further, return the entire statement as one atomic claim.

###
Here are some examples:
Statements: The Green New Deal proposed by Rep. Alexandria Ocasio-Cortez aims to achieve net-zero greenhouse gas emissions and create millions of green jobs.
Atomic Claims: ['The Green New Deal was proposed by Rep. Alexandria Ocasio-Cortez.', 'The Green New Deal aims to achieve net-zero greenhouse gas emissions.', 'The Green New Deal aims to create millions of green jobs.']
Statements: The Inflation Reduction Act, signed into law by President Biden in August 2022, is the largest investment in climate change mitigation in U.S. history.
Atomic Claims: ['The Inflation Reduction Act was signed into law by President Biden in August 2022.', 'The Inflation Reduction Act is the largest investment in climate change mitigation in U.S. history.']
Statements: President Trump's policies led to the highest inflation rate in over 40 years, resulting in economic hardship for millions of American families.
Atomic Claims: ["President Trump's policies led to the highest inflation rate in over 40 years.", "The high inflation rate caused by President Trump's policies resulted in economic hardship for millions of American families."]
###

You will only respond with the atomic claims in the format of a single, one-dimensional list square-brackets of string objects.
Do not provide any explanations or notes.

<<<
Statements: {statements}
>>>
Atomic Claims: []"""

def generate_atomic_claims(statements, num_examples=3):
  """
  Generates atomic claims for the input statements.

  Args:
      claim (str): The input statements.
      num_examples (int, optional): The number of few-shot examples to include in the prompt. Defaults to 3.

  Returns:
      str: The generated atomic claims.
  """
  if num_examples > 0: # Populate the prompt with few-shot examples (w/ proper formatting)
      examples_text = ""
      best_examples = select_best_examples(statements, examples["claim_atomization_examples"], "statement", num_examples)

      # Add each example to the prompt
      for example in best_examples:
          examples_text += f"Statements: {example['statement']}\n"
          examples_text += f"Atomic Claims: {example['atomic_claims']}\n"

      # Finally, fill in the prompt template with the examples and the input statements
      prompt = claim_atomization_template.format(examples=examples_text, statements=statements)
  else: # Otherwise leave the examples section of the prompt template blank and only include the input statements
      prompt = claim_atomization_template.format(examples="", statements=statements)

  #
  response = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=[
      {"role": "system",
      "content": "You are a helpful assistant dividing up a statement into independent claims."}, # type task
      {"role": "user",
      "content": prompt} # prompt
    ],
    temperature = 0
  )
  # Decode the generated text
  output_text = response.choices[0].message.content
  #return output_text
  # Extract only the list of claims from the model's output
  try:
      # Assuming output format directly returns Python list
      atomic_claims = eval(output_text.split('Atomic Claims:')[-1].strip())
      # POST-PROCESSING ERROR HANDLING: If list contains lists, return a flattened list
      if isinstance(atomic_claims[0], list):
          atomic_claims = [item for sublist in atomic_claims for item in sublist]
      return atomic_claims
  except:
      print(f"Error parsing model output: {output_text}")
      return ["Error parsing model output"]


statements = '''
"In New York, there are no barriers to law enforcement to work with the federal government on immigration laws, and there are 100 crimes where migrants can be handed over."
'''
atomic_claims = generate_atomic_claims(statements, 0)
print(f'statement: {statements}')
print(f'atomic claims: {atomic_claims}')


In [ ]:
## Question Generation

# Define prompt template
question_generation_template = """
You are a helpful assistant. Let's break down the claim to questions step by step.
Your task is to provide a set of unique, independent questions to search on the web to verify the claim given after <<<>>>.

Question generation criteria:
1. Each question should be related to the claim, context-independent, and be understood without access to the claim.
2. Each question should be able to be fact-checked by a True/False.
3. Be as specific and concise as possible. Try to minimize the number of questions.
4. Include enough details (e.g., pronoun specification, pronoun disambiguation) to ensure that the claim can be verified.

###
Here are some examples:
Claim: The high inflation rate caused by President Trump's policies resulted in economic hardship for millions of American families.
Questions: ["How did the high inflation rate under President Trump's policies affect American families?", "What economic challenges did millions of American families face due to the inflation rate under President Trump's policies?", 'What are the consequences of economic hardship caused by inflation under President Trump?']
Claim: President Trump's policies led to the highest inflation rate in over 40 years.
Questions: ["What impact did President Trump's policies have on the inflation rate?", "How does the inflation rate during President Trump's term compare to previous years?", "What is the significance of the inflation rate under President Trump's policies?"]
###

You will only respond with the generated questions in the format of a single, one-dimensional list in square-brackets. Do not provide any explanations or notes.

###
Here are some examples:
{examples}###

<<<
Claim: {claim}
>>>
Questions: []"""

def generate_questions(claim, num_examples=3):
    """
    Generates questions to verify the factuality of the input claim.

    Args:
        claim (str): The input claim.
        num_examples (int, optional): The number of few-shot examples to include in the prompt. Defaults to 3.

    Returns:
        str: The generated questions.
    """
    if num_examples > 0: # Populate the prompt with few-shot examples (w/ proper formatting)
        examples_text = ""
        best_examples = select_best_examples(claim, examples["question_generation_examples"], "claim", num_examples)

        # Add each example to the prompt
        for example in best_examples:
            examples_text += f"Claim: {example['claim']}\n"
            examples_text += f"Questions: {example['questions']}\n"

        # Finally, fill in the prompt template with the examples and the input claim
        prompt = question_generation_template.format(examples=examples_text, claim=claim)
    else: # Otherwise leave the examples section of the prompt template blank and only include the input claim
        prompt = question_generation_template.format(examples="", claim=claim)

    # Print the entire prompt for debugging purposes
    # print(prompt)

    # gpt output
    response = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=[
      {"role": "system",
      "content": "You are a helpful assistant dividing up a statement into independent claims."}, # type task
      {"role": "user",
      "content": prompt} # prompt
    ],
    temperature = 0
    )

    # Decode the generated text
    output_text = response.choices[0].message.content

    # Extract only the list of questions from the model's output
    try:
        # Assuming output format directly returns Python list
        questions = eval(output_text.split('Questions:')[-1].strip())
        return questions
    except:
        print(f"Error parsing model output: {output_text}")
        return ["Error parsing model output"]

# Example usage for question generation
claim_question = dict()
for i, claim in enumerate(atomic_claims):
  questions = generate_questions(claim, 0)
  claim_question[claim] = questions
  print(f"Claim: {claim}")
  print(f"Questions: {claim_question[claim]}")


In [ ]:
## Web Querying & Scraping
import json
import requests
import pprint
import re
from bs4 import BeautifulSoup

def extract_website_name(title):
    """Extracts the website name from a given title using regex"""
    delimiters = [' \| ', ' - ']
    for delimiter in delimiters:
        parts = re.split(delimiter, title)
        if len(parts) > 1:
            return parts[-1].strip()  # Return the last part of the split title
    # If no delimiters are found, return the original title
    return title.strip()

def scrape_text_from_website(url):
    """Scrapes text and metadata from a given website URL."""
    try:
        response = requests.get(url, timeout=5)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')

            # Remove script and style tags
            for script in soup(["script", "style"]):
                script.decompose()

            # Extract all text from the website
            text = soup.get_text()

            # Clean up whitespace
            text = re.sub(r'\s+', ' ', text).strip()

            return text
        else:
            print(f"Failed to retrieve content from the URL: {url}")
            return None
    except Exception as e:
        print(f"Error during website scraping: {e}")
        return None

def fetch_search_results(question, scrape_website=False):
    """
    Fetches search results for a given question using an API.

    Args:
        question (str): The question to search for.
        scrape_website (bool, optional): Whether to scrape the website content. Defaults to False.

    Returns:
        list: A list of organic search results.
    """
    api_key = "e9b35305c3b0a79189b7c2dc4c37adbc587d1e65"  # Replace with your actual API key

    headers = {
        "X-API-KEY": api_key,
        "Content-Type": "application/json",
    }

    payload = json.dumps({"q": question})
    try:
        response = requests.post("https://google.serper.dev/search", headers=headers, data=payload)
        result = json.loads(response.text)

        # Extract the organic search results and transform them into our desired format
        results = []
        for item in result['organic']:
            results.append({
                "title": item.get('title', ''),
                "source_title": extract_website_name(item.get('title', '')),
                "date_published": item.get('date', ''),
                "relevant_excerpt": item.get('snippet', ''),
                "text": scrape_text_from_website(item.get('link', '')) if scrape_website else '',
                "search_position": item.get('position', -1),
                "url": item.get('link', ''),
            })

        return results

    except Exception as e:
        print(f"Failed to fetch information: {e}")
        return []

# Example usage:
question = question = '''
In New York, are there barriers to law enforcement to work with the federal government on immigration laws?
'''
search_results = fetch_search_results(question)
search_results

# Pretty print the results
# pprint.pprint(search_results)

In [ ]:
## RAG Retriever
from langchain.docstore.document import Document
from langchain.vectorstores import Chroma
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
import torch

import copy

# Initialize embedding model for retrieval (sentence similarity)
BATCH_SIZE = 32
device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
retriever_model_id='sentence-transformers/all-MiniLM-L6-v2'
retriever_model = HuggingFaceEmbeddings(
    model_name=retriever_model_id,
    model_kwargs={'device': device},
    encode_kwargs={'device': device, 'batch_size': BATCH_SIZE},
)

def retrieve_relevant_documents_using_rag(search_results, content_key, question, chunk_size=512, chunk_overlap=128, top_k=10):
    """
    Takes in search results and a query question, processes and splits the documents,
    and retrieves relevant documents using a RAG approach.

    Args:
        search_results (list of dict): A list of dictionaries containing web-scraped data.
        question (str): The query question for retrieving relevant documents.
        chunk_size (int): The maximum size of the text chunks.
        chunk_overlap (int): The overlap between consecutive text chunks.
        content_key (str): The key in the dictionary containing the text content.
        top_k (int): The number of relevant documents to retrieve.

    Returns:
        list: A list of relevant document chunks.
    """
    # Create LangChain documents from search results
    documents = []
    for result in search_results:
        page_content = result.pop(content_key, None)  # Extract the text content, remaining keys are metadata
        if page_content is not None:
            documents.append(Document(page_content=page_content, metadata=result))

    # Split documents into smaller chunks (if needed, based on document size)
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
    )
    split_documents = text_splitter.split_documents(documents)

    # Initialize ChromaDB vector store to index the document chunks
    db = Chroma.from_documents(
        documents=split_documents,
        embedding=retriever_model,
    )

    # Retrieve the most relevant chunks for the given question
    relevant_docs = db.max_marginal_relevance_search(question, k=top_k)

    return relevant_docs

question = '''
In New York, are there barriers to law enforcement to work with the federal government on immigration laws?
'''
relevant_docs = retrieve_relevant_documents_using_rag(copy.deepcopy(search_results), 'relevant_excerpt', question)
relevant_docs

In [ ]:
relevant_docs

In [ ]:
## RAG-based Question Answering

# Define prompt template
answer_synthesis_template = """
You are a helpful assistant. Let's think step by step. Your task is to synthesize the documents (along with their source metadata) provided below to answer the question given after <<<>>>.

Answer criteria:
1. Start your output with "Answer:"
2  Only use the documents below to answer the question.
3. Cite the relevant documents as JSON (including the source URL) after your answer starting with "Sources:"

If you cannot answer the question given the relevant documents, just say that you don't have enough information to answer the question. Do not make up an answer or sources.

Here are the relevant documents:
{documents}

<<<
Question: {question}
>>>
Answer: """

def synthesize_answer(relevant_docs, question, return_sources=True):
    """
    Synthesizes an answer to a given question using the relevant documents.

    Args:
        relevant_docs (list of dict): A list of relevant document chunks.
        question (str): The question to answer.

    Returns:
        str: The synthesized answer.
    """
    # Format the relevant documents for the prompt
    documents_text = ""
    for doc in relevant_docs:
        documents_text += f"Title: {doc.metadata.get('source_title', '')}\n"
        documents_text += f"URL: {doc.metadata.get('url', '')}\n"
        documents_text += f"Text: {doc.page_content.strip()}\n\n"
        documents_text += f"Date Published: {doc.metadata.get('date_published', '')}\n"

    # Fill in the prompt template with the relevant documents and the question
    prompt = answer_synthesis_template.format(documents=documents_text, question=question)
    # print('*'*20)
    # print(documents_text)
    # print('*'*20)
    prompt = prompt.replace('\n\n', '\n')

    # Print the entire prompt for debugging purposes
    # print(prompt)
    # print('-'*20)

    # gpt output
    response = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=[
      {"role": "system",
      "content": "You are a helpful assistant dividing up a statement into independent claims."}, # type task
      {"role": "user",
      "content": prompt} # prompt
    ],
    temperature = 0
    )

    # Decode the generated text
    output_text = response.choices[0].message.content
    # Extract only the answer from the model's output
    try:
        answer = output_text.split('Answer:')[-1].split('Sources:')[0].strip()
        sources = output_text.split('Sources:')[-1].strip()
        if return_sources: return answer, sources
        return answer
    except:
        print(f"Error parsing model output: {output_text}")
        return "Error parsing model output"

In [ ]:
## Claim Classification

# Define prompt template for reasoning and classification
# 1. -- from CRITIC
claim_classification_template = """
You are a logical reasoning assistant.

Given the original claim, a set of questions to help verify the claim, and their answers, reason step-by-step to come to a verdict on whether the claim is true or false. Think step-by-step about your reasoning process.
Return the verdict after "Verdict:" and provide a clear explanation after "Reasoning:"
For the verdict, only classify the claim as "True", "False", or "Unverifiable."

Reasoning Criteria:
1. Reason the claim over both plausibility and truthfulness.
2. Make sure to only reference to the question and answer pairs for your explanations.

Claim: {claim}

{questions_and_answers}

Verdict: """

def classify_claim(claim, questions, answers, return_reasoning=True):
    """
    Uses a chain-of-thought approach to classify the original claim as true or false based on the answers to generated questions.

    Args:
        claim (str): The original claim.
        questions (list): List of questions related to the claim.
        answers (list): List of answers corresponding to the questions.

    Returns:
        str: The conclusion whether the claim is true or false with reasoning.
    """
    # Format the questions and answers into a single string
    questions_and_answers = ""
    for question, answer in zip(questions, answers):
        questions_and_answers += f"Question: {question}\nAnswer: {answer}\n\n"

    # Fill in the prompt template with the claim and formatted questions and answers
    prompt = claim_classification_template.format(claim=claim, questions_and_answers=questions_and_answers)

    # Print the entire prompt for debugging purposes
    # print(prompt)

    # gpt output
    response = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=[
      {"role": "system",
      "content": "You are a helpful assistant dividing up a statement into independent claims."}, # type task
      {"role": "user",
      "content": prompt} # prompt
    ],
    temperature = 0
    )

    # Decode the generated text
    output_text = response.choices[0].message.content

    # Extract the verdict and reasoning separately from the model's output
    try:
        verdict = output_text.split('Verdict:')[-1].split('Reasoning:')[0].strip()
        reasoning = output_text.split('Reasoning:')[-1].strip()
        if return_reasoning: return verdict, reasoning
        return verdict
    except:
        raise ValueError(f"Error parsing model output: {output_text}")

    return output_text, verdict

In [ ]:
from collections import Counter
import numpy as np
def generate_fact_score(verdicts):
    label = None
    perc_unverified = 0
    v_cleaned = verdicts
    if 'Unverifiable' in verdicts:
        v_cleaned = verdicts[:]
        v_cleaned.remove('Unverifiable')
        perc_unverified = Counter(verdicts)['Unverifiable'] / len(verdicts)
    perc_true = Counter(verdicts)['True'] / len(verdicts)
    perc_false = Counter(verdicts)['False'] / len(verdicts)
    perc = [perc_true, perc_false, perc_unverified]
    winner = np.argwhere(perc == np.amax(perc))
    # print(perc, winner)

    if len(winner) == 3: # three-way tie
        label = "Unverifiable"

    elif len(winner) == 2: # two-way tie
        if 0 in winner and 1 in winner: # half true
            label = 'Half True'
        elif 0 in winner and 2 in winner: # true & unverifable
            label = "Unverifiable"
        elif 1 in winner and 2 in winner: # false & unverifable
            label = "Unverifiable"

    elif len(winner) == 1:
        if 0 in winner:
            if perc_true == 1: # all true
                label = "True"
            elif Counter(v_cleaned)['True'] / len(v_cleaned) > 0.5: # mostly true
                label = "Mostly True"
        elif 1 in winner:
            if perc_false == 1: # all false
                label = "Pants on Fire"
            elif Counter(v_cleaned)['False'] / len(v_cleaned) > 0.5: # mostly false
                label = "Mostly False"
        elif 2 in winner:
            label = 'Unverifiable'
    return label

In [ ]:
def verify_statement_gpt(statement):
    atomic_claims = generate_atomic_claims(statement,0)
    print(len(atomic_claims))
    output_dict = []
    verdicts = []
    for i, claim in enumerate(atomic_claims):
        claim_output = {}
        claim_output['claim'] = claim
        questions = generate_questions(claim,0)
        claim_output['qa-pairs'] = {}
        claim_output['qa-pairs']['questions'] = questions
        answers = []
        sources = []
        for j, question in enumerate(questions):
            print(question)
            search_results = fetch_search_results(question,0)
            relevant_docs = retrieve_relevant_documents_using_rag(search_results, 'relevant_excerpt', question)
            answer, source = synthesize_answer(relevant_docs, question)
            # output[f'claim_{i}']['questions'][f'answer_{j}'] = question
            answers.append(answer)
            sources.append(source)
        claim_output['qa-pairs']['answers'] = answers
        claim_output['qa-pairs']['sources'] = sources
        verdict, reasoning = classify_claim(claim, questions, answers)
        verdicts.append(verdict)
        claim_output['verdict'] = verdict
        claim_output['reasoning'] = reasoning
        output_dict.append(claim_output)
    return verdicts, output_dict, generate_fact_score(verdicts),

In [ ]:
path = '/content/drive/MyDrive/courses/AY 23-24/24 SP/COMS-W3997 LLM Foundations & Ethics/LLM Project/Code/data/pilot.csv'
import pandas as pd
df = pd.read_csv(path)

In [ ]:
df['statement'] = df['statement'].astype(str)

In [15]:
fact_check_samples = []
for i in df['statement'][:10]:
    fact_check_samples.append(verify_statement_gpt(i))

2
What percentage of Generation Z supports Hamas?


Is the support for Hamas among Generation Z evenly split?
What percentage of Generation Z supports Israel?
Is the support for Israel among Generation Z evenly split?
1
Has the trade deficit with China increased during President Trump's tenure?
What was the trend in the trade deficit between the US and China during President Trump's administration?
3
What are the details of the new abortion ban affecting millions of Arizonans?
How will the new abortion ban in Arizona impact the lives of its residents?
What makes the new abortion ban in Arizona considered extreme and dangerous?
Does the abortion ban provide protections for women when their health is at risk?
What are the health risks for women under the abortion ban?
How does the abortion ban address situations where a woman's health is endangered?
Does the abortion ban provide protections for women in cases of rape or incest?
What are the implications of the abortion ban on women involved in tragic cases of rape or incest?
Error parsing

KeyboardInterrupt: 

In [ ]:
gpt4_sample = pd.DataFrame(data = {
    "verdicts": [x[0] for x in fact_check_samples],
    "fact_score": [x[2] for x in fact_check_samples],
    "output": [x[1] for x in fact_check_samples]})

In [ ]:
gpt4_sample.to_excel('/content/drive/MyDrive/courses/AY 23-24/24 SP/COMS-W3997 LLM Foundations & Ethics/LLM Project/gpt4-samples.xlsx')